In [8]:
import pandas as pd
import numpy as np # linear algebra
%matplotlib inline

In [3]:
df = pd.read_json (r'/opt/models/nlp-biases/squad20/train-v2.0.json')

In [4]:
df.head

<bound method NDFrame.head of     version                                               data
0      v2.0  {'title': 'Beyoncé', 'paragraphs': [{'qas': [{...
1      v2.0  {'title': 'Frédéric_Chopin', 'paragraphs': [{'...
2      v2.0  {'title': 'Sino-Tibetan_relations_during_the_M...
3      v2.0  {'title': 'IPod', 'paragraphs': [{'qas': [{'qu...
4      v2.0  {'title': 'The_Legend_of_Zelda:_Twilight_Princ...
..      ...                                                ...
437    v2.0  {'title': 'Infection', 'paragraphs': [{'qas': ...
438    v2.0  {'title': 'Hunting', 'paragraphs': [{'qas': [{...
439    v2.0  {'title': 'Kathmandu', 'paragraphs': [{'qas': ...
440    v2.0  {'title': 'Myocardial_infarction', 'paragraphs...
441    v2.0  {'title': 'Matter', 'paragraphs': [{'qas': [{'...

[442 rows x 2 columns]>

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 2 columns):
version    442 non-null object
data       442 non-null object
dtypes: object(2)
memory usage: 7.0+ KB


In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json # to read json
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

In [12]:
def squad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx
    main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [13]:
# training data
input_file_path = 'train-v2.0.json'
record_path = ['data','paragraphs','qas','answers']
train = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path)

Reading the json file
processing...
shape of the dataframe is (130319, 6)
Done


In [14]:
train.head()

,index,question,context,text,answer_start,c_id
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526.0,0
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166.0,0
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0


In [15]:
train.tail()

,index,question,context,text,answer_start,c_id
130314,5a7e070b70df9f001a875439,Physics has broadly agreed on the definition o...,"The term ""matter"" is used throughout physics i...",NaN,NaN,19028
130315,5a7e070b70df9f001a87543a,Who coined the term partonic matter?,"The term ""matter"" is used throughout physics i...",NaN,NaN,19028
130316,5a7e070b70df9f001a87543b,What is another name for anti-matter?,"The term ""matter"" is used throughout physics i...",NaN,NaN,19028
130317,5a7e070b70df9f001a87543c,Matter usually does not need to be used in con...,"The term ""matter"" is used throughout physics i...",NaN,NaN,19028
130318,5a7e070b70df9f001a87543d,What field of study has a variety of unusual c...,"The term ""matter"" is used throughout physics i...",NaN,NaN,19028


In [20]:
train.count()

index           130319
question        130319
context         130319
text             86821
answer_start     86821
c_id            130319
dtype: int64

In [21]:
train.shape

(130319, 6)

In [22]:
dfqtext=train.filter(["question", "context", "text"]) 

In [23]:
dfqtext.shape

(130319, 3)

In [24]:
dfqtext = dfqtext.dropna()

In [25]:
dfqtext.shape

(86821, 3)

In [26]:
dfqtext.head

<bound method NDFrame.head of                                                  question  \
0                When did Beyonce start becoming popular?   
1       What areas did Beyonce compete in when she was...   
2       When did Beyonce leave Destiny's Child and bec...   
3           In what city and state did Beyonce  grow up?    
4              In which decade did Beyonce become famous?   
...                                                   ...   
130046  In what US state did Kathmandu first establish...   
130047               What was Yangon previously known as?   
130048  With what Belorussian city does Kathmandu have...   
130049  In what year did Kathmandu create its initial ...   
130050                      What is KMC an initialism of?   

                                                  context  \
0       Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
1       Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
2       Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b

In [30]:
dfqtext.loc[0]

question             When did Beyonce start becoming popular?
context     Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
text                                        in the late 1990s
Name: 0, dtype: object

In [36]:
dfqtext.iloc[0,1]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [37]:
dfqtext.nunique()

question    86769
context     18877
text        64763
dtype: int64

In [43]:
dfqtext[dfqtext['context'].str.contains("China")]

,question,context,text
214,Beyonce would take a break from music in which...,Beyoncé announced a hiatus from her music care...,2010
215,Which year did Beyonce and her father part bus...,Beyoncé announced a hiatus from her music care...,2010
216,Which famous landmark did Beyonce see in China?,Beyoncé announced a hiatus from her music care...,the Great Wall of China
217,In what year did Beyonce have her hiatus?,Beyoncé announced a hiatus from her music care...,2010
218,Who inspired this hiatus?,Beyoncé announced a hiatus from her music care...,her mother
...,...,...,...
130046,In what US state did Kathmandu first establish...,"Kathmandu Metropolitan City (KMC), in order to...",Oregon
130047,What was Yangon previously known as?,"Kathmandu Metropolitan City (KMC), in order to...",Rangoon
130048,With what Belorussian city does Kathmandu have...,"Kathmandu Metropolitan City (KMC), in order to...",Minsk
130049,In what year did Kathmandu create its initial ...,"Kathmandu Metropolitan City (KMC), in order to...",1975


In [44]:
dfqtext.iloc[214,1]

'Beyoncé announced a hiatus from her music career in January 2010, heeding her mother\'s advice, "to live life, to be inspired by things again". During the break she and her father parted ways as business partners. Beyoncé\'s musical break lasted nine months and saw her visit multiple European cities, the Great Wall of China, the Egyptian pyramids, Australia, English music festivals and various museums and ballet performances.'

In [68]:
dfqtext[dfqtext['context'].str.contains("Grey")].count()

question    82
context     82
text        82
dtype: int64

In [69]:
dfqtext[dfqtext['context'].str.contains("man")]

,question,context,text
15,"After her second solo album, what other entert...",Following the disbandment of Destiny's Child i...,acting
16,Which artist did Beyonce marry?,Following the disbandment of Destiny's Child i...,Jay Z
17,"To set the record for Grammys, how many did Be...",Following the disbandment of Destiny's Child i...,six
18,For what movie did Beyonce receive her first ...,Following the disbandment of Destiny's Child i...,Dreamgirls
19,When did Beyonce take a hiatus in her career a...,Following the disbandment of Destiny's Child i...,2010
...,...,...,...
130046,In what US state did Kathmandu first establish...,"Kathmandu Metropolitan City (KMC), in order to...",Oregon
130047,What was Yangon previously known as?,"Kathmandu Metropolitan City (KMC), in order to...",Rangoon
130048,With what Belorussian city does Kathmandu have...,"Kathmandu Metropolitan City (KMC), in order to...",Minsk
130049,In what year did Kathmandu create its initial ...,"Kathmandu Metropolitan City (KMC), in order to...",1975


# Liguistic Complexity of Squad 2.0 DataSet

#### Language mastery by humans is not an easy challenge and it is evident in the reading comprehension scores of majority of the school kids. 

#### Similarly, ability to comprehend daily spoken topics doesn't imply you are able to comprehend more abstract and specialized concepts.

#### SQuAD 2.0 dataset is completely produced from 10K popular articles from Wikipedia. Performance of NLP models trained against this dataset 

In [89]:
# Recognizing the main idea in complex readings helps you to write better essays in that it requires you to unpack, 
# understand, and synthesize the thoughts of others. This, in turn, helps you to form your own arguments supported by sound 
# reasoning. The identification of bias is important because it has the potential to undermine your reasoning. Pointing out 
# and explaining any bias in complex readings develops an awareness of bias so you can avoid it in your own writing, 
# which serves to strengthen your arguments.

In [70]:
dfqtext[dfqtext['context'].str.contains("sports")]

,question,context,text
692,House of Dereon became known through Beyonce a...,Beyoncé and her mother introduced House of Der...,her mother
693,Beyonce's grandma's name was?,Beyoncé and her mother introduced House of Der...,Agnèz Deréon
694,Beyonce's family's company name is what?,Beyoncé and her mother introduced House of Der...,Beyond Productions
695,What types of garments are sold by Beyonce's c...,Beyoncé and her mother introduced House of Der...,"sportswear, denim offerings with fur, outerwea..."
696,Which two countries can you purchase Beyonce's...,Beyoncé and her mother introduced House of Der...,US and Canada
...,...,...,...
130031,"Along with cricket, what sport is highly popul...",Football and Cricket are the most popular spor...,Football
130032,What body oversees soccer in Nepal?,Football and Cricket are the most popular spor...,All Nepal Football Association
130033,How many people can fit in Dasarath Rangasala ...,Football and Cricket are the most popular spor...,"25,000"
130034,In what part of Kathmandu is Dasarath Rangasal...,Football and Cricket are the most popular spor...,Tripureshwor


In [73]:
dfqtext.iloc[696,1]

"Beyoncé and her mother introduced House of Deréon, a contemporary women's fashion line, in 2005. The concept is inspired by three generations of women in their family, the name paying tribute to Beyoncé's grandmother, Agnèz Deréon, a respected seamstress. According to Tina, the overall style of the line best reflects her and Beyoncé's taste and style. Beyoncé and her mother founded their family's company Beyond Productions, which provides the licensing and brand management for House of Deréon, and its junior collection, Deréon. House of Deréon pieces were exhibited in Destiny's Child's shows and tours, during their Destiny Fulfilled era. The collection features sportswear, denim offerings with fur, outerwear and accessories that include handbags and footwear, and are available at department and specialty stores across the US and Canada."

In [74]:
dfqtext[dfqtext['context'].str.contains("cancer")]

,question,context,text
6673,Hypersexual behavior in male dogs is diminishe...,Neutering reduces problems caused by hypersexu...,Neutering
6674,"According to the text, what is a possible side...",Neutering reduces problems caused by hypersexu...,urinary incontinence
6675,Female dogs are less likely to develop cancer ...,Neutering reduces problems caused by hypersexu...,Spayed
6676,What is increased in female dogs with neutering?,Neutering reduces problems caused by hypersexu...,urinary incontinence
6677,What is increased in male dogs with neutering?,Neutering reduces problems caused by hypersexu...,prostate cancer
...,...,...,...
128983,When does infection begin?,Infection begins when an organism successfully...,"when an organism successfully enters the body,..."
128984,What group is not easily infected?,Infection begins when an organism successfully...,humans
128985,What group of humans have increased susceptibi...,Infection begins when an organism successfully...,"weak, sick, malnourished, have cancer or are d..."
128986,What individuals are particularly susceptible ...,Infection begins when an organism successfully...,Individuals who have a suppressed immune system


In [75]:
dfqtext[dfqtext['question'].str.contains("cancer")]

,question,context,text
6675,Female dogs are less likely to develop cancer ...,Neutering reduces problems caused by hypersexu...,Spayed
6892,How many cancer patients has Daisy detected?,Medical detection dogs are capable of detectin...,551
19015,What cancer can be caused by too much time in ...,DST has mixed effects on health. In societies ...,skin cancer
31383,How does MD Anderson Cancer Center rank in US ...,The Baylor College of Medicine has annually be...,one of the top two
47949,Who announced that SV40 was not associated wit...,"During the years 1940-1955, the rate of declin...",United States Cancer Institute
68263,In which country was a study about the presenc...,"Heart disease, cancer, obesity, and diabetes a...",China
68347,Which organization did a study on cancer in de...,Cancer is now common in developing countries. ...,International Agency for Research on Cancer
68348,What were cancers such as liver cancer or stom...,Cancer is now common in developing countries. ...,consumption of carcinogenic preserved foods
68349,The rise of lung cancer in poor nations can be...,Cancer is now common in developing countries. ...,tobacco
68350,"Obesity, lack of exercise, age and diet are ri...",Cancer is now common in developing countries. ...,Western


In [84]:
dfqtext[dfqtext['context'].str.contains("math")]

,question,context,text
3096,When did an investigation occur under school c...,Details of school casualties had been under no...,December 2008
3097,When was the official tally of students killed...,Details of school casualties had been under no...,"May 7, 2009"
3098,How many students were killed in Xinhua?,Details of school casualties had been under no...,"5,335"
3099,How many students were disabled in Xinhua?,Details of school casualties had been under no...,546
3100,What date were the official numbers of student...,Details of school casualties had been under no...,"May 7, 2009"
...,...,...,...
127720,Which president at Notre Dame created the Coll...,The College of Science was established at the ...,Father Patrick Dillon
127721,In what year was the Notre Dame College of Sci...,The College of Science was established at the ...,1865
127722,How many years long was a scientific course un...,The College of Science was established at the ...,six years
127723,Which hall at Notre Dame contains the current ...,The College of Science was established at the ...,Jordan Hall of Science


In [86]:
dfqtext.iloc[3100,0]

'The English occupied New York City beginning on what date?'

In [88]:
dfqtext[dfqtext['question'].str.contains("Math")]

,question,context,text
13023,Where is The Oklahoma School of Science and Ma...,The Oklahoma School of Science and Mathematics...,Oklahoma City
18652,"Along with Frank Sinatra, Doris Day, Johnny Ma...","Initially, the vocalists consisted of artists ...",Connie Francis
18781,"Along with Andy Williams, Johnny Mathis, Nana ...",Artists contributing to this format include ma...,Frank Sinatra
45501,In what year was Issaac Newton's Principia Mat...,There is little consensus on the precise begin...,1687
45808,Who wrote the celebrated Philosophiae Naturali...,One of the most important developments that th...,Sir Isaac Newton
71706,How many theories were included within the Pri...,"In 1687, Isaac Newton published the Principia ...",two
105856,During what era was the Nine Chapters on the M...,Three Han mathematical treatises still exist. ...,Han
105862,What were positive numbers portayed as in the ...,One of the Han's greatest mathematical advance...,red counting rods
114668,Who co-wrote Principia Mathematica with Whiteh...,In addition to numerous articles on mathematic...,Bertrand Russell
114672,With what mathematician and philosopher did Wh...,In addition to numerous articles on mathematic...,Bertrand Russell
